In [63]:
import glob
import numpy as np
import random
import librosa
from sklearn.model_selection import train_test_split
import librosa
import numpy as np
import pdb
import string
from Levenshtein import distance
import glob
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from scipy import stats

In [64]:
####################################################################  Utilis ###############################################################
def wav2feat(wavfile):
    '''
    Input: audio wav file name
    Output: Magnitude spectrogram
    '''
    x, Fs = librosa.load(wavfile, sr=44100, mono=True) 
    hop = int(0.01 * Fs) # 10ms
    win = int(0.02 * Fs) # 20ms
    X = librosa.stft(x, n_fft=1024, hop_length=hop, win_length=win, window='hann', center=True, pad_mode='reflect')
    return np.abs(X)

def wavs2feat(wavfiles):
    '''
    Concatenate the audio files listed in wavfiles
    Input: list of audio wav file names
    Output: Magnitude spectrogram of concatenated wav
    '''
    x = []
    for wf in wavfiles:
        x1, Fs = librosa.load(wf, sr=44100, mono=True)
        x.append(x1)
    x = np.hstack(x)
    hop = int(0.01 * Fs) # 10ms
    win = int(0.02 * Fs) # 20ms
    X = librosa.stft(x, n_fft=1024, hop_length=hop, win_length=win, window='hann', center=True, pad_mode='reflect')
    return np.abs(X)

def read_csv(filename):
    id_label = {}
    with open(filename,'r') as fid:
        for line in fid: # '176787-5-0-27.wav,engine_idling\n'
            tokens = line.strip().split(',') # ['176787-5-0-27.wav', 'engine_idling']
            id_label[tokens[0]] = tokens[1]
    return id_label

def editDistance(gt, est):
    '''both are lists of labels
    E.g. gt is "dog_bark-street_music-engine_idling"
    E.g. est is "street_music-engine_idling"
    '''
    gttokens = gt.split('-')
    esttokens = est.split('-')
    # Map token to char
    tokenset = list(set(gttokens+esttokens)) # ['dog_bark', 'siren', 'street_music', 'engine_idling']
    token_char = {}
    for i in range(len(tokenset)):
        token_char[tokenset[i]] = string.ascii_uppercase[i]  # {'dog_bark': 'A', 'siren': 'B', 'street_music': 'C', 'engine_idling': 'D'}
    # convert gt and est to strings
    gtstr = [token_char[t] for t in gttokens]
    gtstr = ''.join(gtstr)  # 'BCA'
    eststr = [token_char[t] for t in esttokens]
    eststr = ''.join(eststr)  # 
    # Compare
    editdist = distance(gtstr, eststr) # 1
    score = 1 - editdist/len(gtstr)
    return editdist, score

def evals(gtcsv, estcsv, taskid):
    gt_id_label = read_csv(gtcsv)
    est_id_label = read_csv(estcsv)
    score = 0
    for id in est_id_label:
        if taskid==1:
            if est_id_label[id] == gt_id_label[id]:
                score += 1
        elif taskid==2:
            _, ss = editDistance(gt_id_label[id], est_id_label[id])
            score += ss
        else:
            pdb.set_trace()
            assert False, ["taskid not correct; it is", taskid]
    avgScore = score/len(est_id_label)
    return avgScore

############################################################################################################################################

In [82]:
# function for making dict of labels which maps the corresponding class to number 
# function for and dict of numbers which maps the corresponding label

def labels_y(y_csv):
  labels = {}
  labels_ = {}
  j=0
  for i in y_csv:
    if y_csv[i]=='class': continue
    labels[y_csv[i]]=1
  for i in labels:
    labels[i]=j
    j=j+1
  for i in labels:
    labels_[labels[i]]=i

  return  labels, labels_

In [83]:
# function to create data which is to be trained

def make_data(X,y):
  x = []
  Y = []
  for j in range(len(X)):
    for i in range(0, X[j].shape[1]-20, 5):
      x.append(X[j][:, i:i+20])
      Y.append(y[j])

  x=np.array(x)
  Y=np.array(Y)
  x = np.reshape(x,(len(x), 513*20))
  Y=np.reshape(Y, (len(Y),))
  return x,Y

In [85]:
y_csv = read_csv('/content/drive/MyDrive/shared_train/labels_train.csv')

In [86]:
labels, labels_ = labels_y(y_csv)

In [ ]:
f = glob.glob('/content/drive/MyDrive/shared_train/audio_train'+'/*.wav')

In [ ]:
# making dataset to train

X=[]
y=[]
for F in f:
  X.append(wav2feat(F))
  y.append(labels[y_csv[F.split('/')[-1]]])

X_, y_ = make_data(X,y)

In [ ]:
# split data into test and train

X_train, X_test, y_train, y_test = train_test_split( X_, y_, test_size=0.2)

In [ ]:
# traing the model

clf = RandomForestClassifier(n_estimators = 30)
clf.fit(X_train, y_train)

print ('fit to train new: ', clf.score(X_train, y_train))
print ('fit to test: ', clf.score(X_test, y_test))

fit to train new:  0.9999586161231584
fit to test:  0.9473618870266914


In [ ]:
# saving the model for future use

import pickle
with open('/content/drive/MyDrive/test_2.pickle', "wb") as f:
     pickle.dump(clf, f)

In [67]:
# loading the model

import pickle
with open('/content/drive/MyDrive/test_2.pickle', "rb") as f:
     clf = pickle.load(f)

In [91]:
# function of creating final output of numbers by removing repeated numbers

def final(Yt):
  yf = []
  for y in Yt:
    t =[]
    i=0
    a = y[i]
    b = 1
    for i in range(1, len(y)):
      if y[i]==a: b=b+1
      else :
        if b>7:
          t.append(a)
        a=y[i]
        b=1
      if len(t)>1 and t[len(t)-1]==t[len(t)-2]: t=t[:len(t)-1]
      if i==len(y)-1 and b>7 : t.append(a)
    if len(t)>1 and t[len(t)-1]==t[len(t)-2]: t=t[:len(t)-1]
    if len(t)==0: t.append(stats.mode(y))
    yf.append(t)
  return yf


  # function of converting it to string of labels
def labels_final(yf, labels):
  l_f = []
  for y in yf:
    str_ = ""
    i=0
    for a in y:
      if i!=0: str_=str_+'-'
      str_ = str_+labels[a]
      i=i+1
    l_f.append(str_)
  return l_f




In [68]:
############################################# testing model for sample_task_2 #########################################################################

# list of file address
f = glob.glob('/content/drive/MyDrive/shared_train/sample_test_task2/feats'+'/*.npy')
f.sort()

# creating list of file names
F_=[]
for a in f:
  F_.append(a.split('/')[-1].split('.')[0])


# loading input and breaking it into parts 
# then creating the output it
Xt = []
Yt = []
for F in f:
  a=np.load(F)
  Xt = []
  for i in range(0, a.shape[1]-5, 5):
    if i+20<a.shape[1]:Xt.append(a[:, i:i+20])
    else: Xt.append(np.pad(a[:,i:],((0,0),(0,20-len(a[:,i:][0])))))
  if len(Xt)==0: Xt.append(np.pad(a[:,0:],((0,0),(0,20-len(a[:,0:][0])))))
  Xt=np.array(Xt)
  Xt = np.reshape(Xt,(len(Xt), 513*20))
  Yt.append(clf.predict(Xt))

 
# creating final output 
Yf = final(Yt)
l_f = labels_final(Yf, labels_)

In [ ]:
# save output to a csv file

import pandas as pd
df = pd.DataFrame(data={"col1": F_, "col2": l_f})
df.to_csv('/content/drive/MyDrive/Certificates/file3.csv', sep=',',index=False,header=False)

In [ ]:
# print result of sample_tets_task_2

print(evals('/content/drive/MyDrive/shared_train/sample_test_task2/labels.csv','/content/drive/MyDrive/Certificates/file3.csv',2))

0.9416666666666668


In [ ]:
####################################################### final task 2 #######################################################################

In [88]:


# list of file address
f = glob.glob('/content/drive/MyDrive/test_task2/feats'+'/*.npy')
f.sort()

# creating list of file names
F_=[]
for a in f:
  F_.append(a.split('/')[-1].split('.')[0])


# loading input and breaking it into parts 
# then creating the output it
Xt = []
Yt = []
for F in f:
  a=np.load(F)
  Xt = []
  for i in range(0, a.shape[1]-5, 5):
    if i+20<a.shape[1]:Xt.append(a[:, i:i+20])
    else: Xt.append(np.pad(a[:,i:],((0,0),(0,20-len(a[:,i:][0])))))
  if len(Xt)==0: Xt.append(np.pad(a[:,0:],((0,0),(0,20-len(a[:,0:][0])))))
  Xt=np.array(Xt)
  Xt = np.reshape(Xt,(len(Xt), 513*20))
  Yt.append(clf.predict(Xt))

 
# creating final output 
Yf = final(Yt)
l_f = labels_final(Yf, labels_)

In [89]:
# save output to a csv file

import pandas as pd
df = pd.DataFrame(data={"col1": F_, "col2": l_f})
df.to_csv('/content/drive/MyDrive/Certificates/file3.csv', sep=',',index=False,header=False)

In [92]:
for l in l_f:
  print(l)

dog_bark-street_music-dog_bark
jackhammer-street_music-drilling-dog_bark
dog_bark-street_music
dog_bark
street_music-dog_bark
street_music-engine_idling-children_playing-street_music
gun_shot-engine_idling-street_music-jackhammer
drilling-engine_idling
dog_bark-street_music-siren
engine_idling-street_music
street_music-dog_bark-drilling
children_playing-engine_idling-siren-dog_bark
dog_bark-drilling
engine_idling
street_music-drilling-dog_bark-street_music
street_music-jackhammer-street_music
siren-jackhammer
street_music-siren
drilling-street_music
street_music-air_conditioner-siren-dog_bark-siren-dog_bark
dog_bark-street_music-children_playing
dog_bark-street_music-drilling
jackhammer-drilling-street_music
drilling-dog_bark
siren-dog_bark-drilling-dog_bark
drilling-street_music-drilling
street_music-engine_idling-street_music-children_playing-street_music
street_music-children_playing-jackhammer
dog_bark-engine_idling-dog_bark
drilling-street_music
